In [ ]:
from ReadData import read_data
import xpress as xp
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from ReadData import interp

df_RE, df_Grid, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods, df_lifetime, df_2MW, df_20MW, df_200MW = read_data()

In [ ]:
def model_v1(df_RE, df_Grid, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods, df_lifetime, df_stack_replacement, Demand, x_FID = 2025, Project_Years = 26, miprelstop = 0.02, maxtime = 120):
    '''
    Solves hydrogen production problem for Wood Mackenzie optimising costs and emissions

    Inputs:
        df_RE: DataFrame with hourly solar, offshore wind, onshore wind data
        df_Grid: DataFrame with hourly CO2 intensity of power from grid and its price
        df_Elctro: DataFrame with Electrolyser parameters for PEM ALK SOEC
        df_Elctro_Costs: DataFrame with CAPEX costs of electrolysers based on different capacities
        df_Battery: DataFrame with Battery storage costs
        df_repperiods: DataFrame with representative periods and their respective beginning and end indexes and weights
        df_lifetime: DataFrame with stack lifetime in years indexed by electrolyser e and year of replacement
        df_stack_replacement: DatFrame with replacement costs of stack indexed by electrolyser name and year
        Demand: Demand in kg
        x_FID: The year of the first investment decision ( default 2025 )
        Project_Years: The project lifetime in years ( default at 26 for 2025 - 2050 )
        df_PPA: DataFrame with PPA prices for Solar, Wind onshore and offshore
        miprelstop: MIP gap at which to terminate (default at 2%)
        maxtime: max time to allow the solver to run for (default 120s)
    
    Outputs:

    '''

    # Notes of things to change:
    # 1. Change stack replacement cost to changing over years and dependant on the year ur replacing it in
    # 2. Change PPA to be variable
    # 3. Add variable efficiencies of electrolyser
    # 4. Change to correct units
    # 5. Objective: CAPEX Costs: PPA + Battery Capacity + H2 Store capacity + Electrolyser Capacity\
                #   OPEX Costs: Stack replacement dependant on diff lifetimes and prices + Power costs for 25 yrs based on rep periods + Fixed opex for battery, store, electrolyser

    xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

    def clean_name(r):
        return r.replace(' ', '_')

    prob = xp.problem(name="Hydrogen WoodMac")


    # ------------ SETS -------------
    T = list(
        df_RE[['Report_Year', 'Report_Month', 'Report_Day', 'Report_Hour']]
        .drop_duplicates()
        .itertuples(index=False, name=None)
    )
    E = list(df_Elctro['Type'].unique())
    R = {col for col in df_RE.columns if not col.startswith('Report_')}
    K = df_repperiods['K'].index

    # ------ DECISION VARIABLES ------

    # Proportion of renewable energy contracted to take from renewable site r
    PPA = {(r): xp.var(vartype=xp.continuous, name = f'PPA_{clean_name(r)}', lb= 0, ub=1) for r in R}

    # Power bought from the grid at time t (kW)
    P_Grid_b = {(y,m,d,h): xp.var(vartype=xp.continuous, name = f'P_Grid_b_{y}_{m}_{d}_{h}') for (y,m,d,h) in T}

    # Power sold to the grid at time t (kW)
    P_Grid_s = {(y,m,d,h): xp.var(vartype=xp.continuous, name = f'P_Grid_s_{y}_{m}_{d}_{h}') for (y,m,d,h) in T}

    # Power taken out of battery at time t (kW)
    P_Bat_out = {(y,m,d,h): xp.var(vartype=xp.continuous, name = f'P_Bat_out_{y}_{m}_{d}_{h}') for (y,m,d,h) in T}

    # Power put into battery at time t (kW)
    P_Bat_in = {(y,m,d,h): xp.var(vartype=xp.continuous, name = f'P_Bat_in_{y}_{m}_{d}_{h}') for (y,m,d,h) in T}

    # Power put into electrolyser e at time t (kW)
    P_Ez = {(e,y,m,d,h): xp.var(vartype=xp.continuous, name = f'P_Ez_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # # Power required for putting H2 into storage at time t (kW)
    # P_H2st = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_{y}_{m}_{d}_{h}') for t in T}

    # # Hydrogen leaving store at time t (kg/h)
    # H_H2st_out = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_out_{y}_{m}_{d}_{h}') for t in T}

    # # Hydrogen entering store at time t (kg/h)
    # H_H2st_in = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_in_{t}') for t in T}

    #  Hydrogen leaving electrolyser e at time t (kg/h)
    H_Ez_out = {(e, y,m,d,h): xp.var(vartype=xp.continuous, name = f'H_Ez_out_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # Energy stored in battery at time t (kWh)
    E_Bat = {(y,m,d,h): xp.var(vartype=xp.continuous, name = f'E_Bat_{y}_{m}_{d}_{h}') for (y,m,d,h) in T}

    # # Hydrogen stored at time t (kg)
    # E_H2st = {(t): xp.var(vartype=xp.continuous, name = f'E_H2st_{t}') for t in T}

    # Energy Capacity of battery (kWh)
    Q_Bat_cap = xp.var(vartype=xp.continuous, name='Q_Bat_cap')

    # # Energy capacity of H2 storage tank (kg)
    # Q_H2st_cap = xp.var(vartype=xp.continuous, name='Q_H2st_cap')

    # Power capacity of electrolyser e (kW)
    P_Ez_cap = {(e): xp.var(vartype=xp.continuous, name = f'P_Ez_cap_{e}') for e in E}

    # Power capacity of battery (kW)
    P_Bat_cap = xp.var(vartype=xp.continuous, name='P_Bat_cap')

    # Load factor of electrolyser e at time t
    Load_Ez = {(e, y,m,d,h): xp.var(vartype=xp.continuous, name = f'Load_Ez_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # Cumulative hours at time t an electrolyser e has been operating for since last stack replacement
    H_Ez_cum = {(e, y,m,d,h): xp.var(vartype=xp.continuous, name = f'H_Ez_cum_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # Binary variable for if electrolyser e is on at time t
    z = {(e, y,m,d,h): xp.var(vartype=xp.binary, name = f'z_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # Binary variable if stack for electrolyser e is replaced at time t
    R = {(e, y,m,d,h): xp.var(vartype=xp.binary, name = f'R_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # Energy needed for electrolyser e per kg of hydrogen output as  a function of the load factor 
    Ez_kWh_per_kg = {(e, y,m,d,h): xp.var(vartype=xp.continuous, name = f'Ez_kWh_per_kg_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}
    
    # The lifetime of current stack at electrolyser e at time t determined by last replacement 
    Life_Stack_current = {(e, y,m,d,h): xp.var(vartype=xp.continuous, name = f'Life_Stack_current_{e}_{y}_{m}_{d}_{h}') for e in E for (y,m,d,h) in T}

    # --------- INDEX DATA ------------
    # ADD CO2 INTENSITY!! 

    # Index RE and Grid data by (year, month, date, hour)
    df_RE.set_index(['Report_Year', 'Report_Month', 'Report_Day', 'Report_Hour'], inplace=True)
    df_Grid.set_index(['Report_Year', 'Report_Month', 'Report_Day', 'Report_Hour'], inplace=True)
    df_PPA_index = df_PPA.copy()
    df_PPA_index.set_index('Renewable Source', inplace=True)

    # Index electrolyser data by electrolyser name
    df_Elctro_index = df_Elctro.copy()
    df_Elctro_index.set_index('Type', inplace=True)

    # Change replacement and lifetime stack data to be indexed by (y,m,d,h) instead of just year 
    df_lifetime_full = {}

    for (e, y) in df_lifetime.index:
        for (y_t, m, d, h) in T:
            if y_t == y: 
                df_lifetime_full[(e, (y, m, d, h))] = df_lifetime.loc[(e, y)]

    df_replacement_full = {}

    for (e, y) in df_stack_replacement.index:
        for (y_t, m, d, h) in T:
            if y_t == y: 
                df_replacement_full[(e, (y, m, d, h))] = df_stack_replacement.loc[(e, y)]

    # --------- PARAMETERS ------------

    # Power available from renewable site r at time t (kW)
    P_PPA = {(r, (y,m,d,h)): df_RE.loc[(y,m,d,h), r] for (y,m,d,h) in T for r in df_RE.columns}

    # Constant on-site daily hydrogen demand (kg)
    D_H2 = Demand

    # Round-trip efficiency for battery
    Bat_eff = df_Battery['Round trip efficiency']

    # CO2 Intensity of power from the grid at time t (kg/kWh)
    Int_Grid = df_Grid['CO2 Intensity (kg CO2/kWh)'].to_dict()

    # Maximum CO2 emissions in kg CO2 e/kg H2
    Int_max = 2.4

    # Min and Max load for electrolyser e
    Ez_min_load = df_Elctro_index['Minimum Load'].to_dict()
    Ez_max_load = df_Elctro_index['Maximum Load'].to_dict()

    # Duration in hours of the battery
    Bat_dur = df_Battery['Duration (hrs)']

    # Number of years over time horizon
    N_years = Project_Years

    # Stack lifetime for electrolyser e in hours if replaced/built in time period t, have to copy all years for all m,d,h
    Life_Stack = df_lifetime_full

    # Coefficients for efficiency calculation of electrolyser e
    x5 = df_Elctro_index['x5'].to_dict()
    x4 = df_Elctro_index['x4'].to_dict()
    x3 = df_Elctro_index['x3'].to_dict()
    x2 = df_Elctro_index['x2'].to_dict()
    x1 = df_Elctro_index['x1'].to_dict()
    x0 = df_Elctro_index['x0'].to_dict()

    # Final Investment Decision Year
    x_FID = x_FID

    # System efficiency degredation 
    x_eff_deg = df_Elctro_index['System Efficiency Degradation'].to_dict()

    # System degredation factor 
    x_deg_fact = {}
    for e in E:
        for (y,m,d,h) in T:
            x_deg_fact[(e, (y,m,d,h))] = x_eff_deg^(x_FID - y)
    
    # PPA Price 
    C_PPA = df_PPA_index['PPA Price (£/kWh)']

    # Cost for buying and selling power from the grid
    C_Grid = df_Grid['Price (£/kWh, real 2025)'].to_dict()

    # CAPEX Cost for capacity of battery store
    Bat_capex = df_Battery['Capex ($/kWh)']

    # CAPEX Cost for electrolyser e
    C_Ez_capex = df_Elctro_Costs['Total Installed Cost (TIC) (£/kW)']

    # Fixed OPEX cost for electrolyser e as proportion of capex
    C_Ez_fixed_opex = df_Elctro_index['Fixed Opex percent'].to_dict()

    # Fixed OPEX cost for battery as proportion of capex
    C_Bat_fixed_opex = df_Battery['Fixed Opex percent']

    # Replacement costs of the stack in electrolyser e if replaced in time period t
    C_Replace = df_replacement_full

    # Costs which make up CAPEX cost of electrolysers
    C_Ez_BoS = df_Elctro_Costs['Balance of Stack (£/kW)']
    C_Ez_BoP = df_Elctro_Costs['Balance of Plant (£/kW)']
    C_Ez_EPC = df_Elctro_Costs['Engineering, Procurement & Construction costs (£/kW)']
    C_Ez_Owners = df_Elctro_Costs['Owners costs (£/kW)']


    # ---------- CONSTRAINTS ------------

    # Power Balance:
    prob.addConstraint( xp.Sum(PPA[r]*P_PPA[r,t] for r in R )+ P_Grid_b[t]+ P_Bat_out[t] == P_Grid_s[t] +P_Bat_in[t] + P_H2st[t] + xp.Sum(P_Ez[e,t] for e in E) for t in T)
    
    # Hydrogen Balance:
    prob.addConstraint( H_H2st_out[t] + xp.Sum(H_Ez_out[e,t] for e in E) == H_H2st_in[t] + D_H2 for t in T)

    # H2 Storage:

    # Electrolyser:

    # Battery:

    # Average CO2 Emissions:

    # Unit Commitment:

    # Stack Replacement:

    # ---------- OBJECTIVE FUNCTION ----------

    # CAPEX Costs:
    # Electrolyser + H2 Storage + Battery based on Capacities
    # capacities = xp.Sum(xp.Sum(PPA[r]*P_PPA[r,t]*C_PPA[r] for r in R) + C_ for t in T)
    # depends on capacity chosen!!!

    # OPEX Costs:
    # Electrolyser + H2 Storage + Battery fixed OPEX based on % of CAPEX

    # Variable OPEX for Power bought and sold on Grid

    # Stack Replacement Costs


    return T,E,R,K,PPA

T,E,R,K,PPA = model_v1(df_RE, df_Grid, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods, miprelstop = 0.02, maxtime = 120)
print(T,E,R,K)
print(PPA)

RangeIndex(start=0, stop=227904, step=1) ['PEMWE', 'AWE', 'SOEC'] {'Solar PV', 'Wind Onshore', 'Wind Offshore'} RangeIndex(start=0, stop=12, step=1)
Renewable Source
Solar PV         0.0490
Wind Onshore     0.0456
Wind Offshore    0.0422
Name: PPA Price (£/kWh), dtype: float64
